## STEP 4: DEVELOP A FUZZY LOGIC MODEL

A fuzzy logic model is one that is built on expert knowledge rather than
training data. You may wish to use the
[`scikit-fuzzy`](https://pythonhosted.org/scikit-fuzzy/) library, which
includes many utilities for building this sort of model. In particular,
it contains a number of **membership functions** which can convert your
data into values from 0 to 1 using information such as, for example, the
maximum, minimum, and optimal values for soil pH.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>To train a fuzzy logic habitat suitability model:</p>
<pre><code>1. Research S. nutans, and find out what optimal values are for each variable you are using (e.g. soil pH, slope, and current climatological annual precipitation). 
2. For each **digital number** in each raster, assign a **continuous** value from 0 to 1 for how close that grid square is to the optimum range (1=optimal, 0=incompatible). 
3. Combine your layers by multiplying them together. This will give you a single suitability number for each square.
4. Optionally, you may apply a suitability threshold to make the most suitable areas pop on your map.</code></pre></div></div>

> **Tip**
>
> If you use mathematical operators on a raster in Python, it will
> automatically perform the operation for every number in the raster.
> This type of operation is known as a **vectorized** function. **DO NOT
> DO THIS WITH A LOOP!**. A vectorized function that operates on the
> whole array at once will be much easier and faster.

<span style='color: purple'>

Optimal and tolerable ranges:

* Soil:
    * Optimal pH: 6.5; range = 6.0 - 7.0
        * *None of the values in the siskiyou_soil_ph_da are exactly 6.5, and I know that redwoods currently live in Siskiyou, so instead of using a triangular membership function w/ one optimum value, we'll use a trapezoidal membership function and a range around 6.5.*
    * Tolerable range: 5.0 - 7.5
    * *Source:* https://www.fs.usda.gov/psw/publications/documents/psw_rp028/psw_rp028.pdf

* Elevation:
    * Optimal: 100 ft - 2,500 ft
    * Tolerable range: sealevel - 3,000 ft
    * *Source for both ranges:* https://www.fs.usda.gov/psw/publications/documents/psw_rp028/psw_rp028.pdf

* Average annual precipitation:
    * Optimal: 50 in - 70 in (1270 mm - 1800 mm)
        * It was difficult to find a description of an exact optimal range of precipitation for the coastal redwood, so I am using the mean precipitation values from two different sources. The 50 inch number is from [here](https://www.nps.gov/parkhistory/online_books/shirley/sec4.htm#:~:text=The%20main%20part%20of%20the,100%20inches%20of%20rainfall%20annually.) and the 70 inch number is from [here](https://www.fs.usda.gov/database/feis/plants/tree/seqsem/all.html#11). 
    * Tolerable range: [20 in - 200 in (508 mm - 5080 mm)](https://plants.usda.gov/plant-profile/SESE3/characteristics)
</span>

<span style='color: purple'>

Load in stored variables:

</span>

In [1]:
%store -r reproj_das_list

<span style='color: purple'>

Import packages:

</span>

In [2]:
import numpy as np # scientific computing
import rioxarray as rxr # Work with raster data
import rioxarray.merge as rxrmerg # merge rasters
import skfuzzy as fuzz # Create fuzzy logic model
from tqdm.notebook import tqdm # Progress bars on loops
import xarray as xr # Work with DataArrays

<span style='color:purple'>

## Habitat suitability Fuzzy Logic for all climate model situations

Going to use the trapezoidal membership function so that our optimal values have a range and our tolerable values have a range

1. Create fuzzy models for each DataArray

2. Complete raster multiplication:

    a. Siskiyou pH fuzzy model * Siskiyou elevation fuzzy model * each of the four Siskiyou 2050 climate fuzzy models

    b. Siskiyou pH fuzzy model * Siskiyou elevation fuzzy model * each of the four Siskiyou 2080 climate fuzzy models
    
    c. Padres pH fuzzy model * Padres elevation fuzzy model * each of the four Padres 2050 climate fuzzy models

    d. Padres pH fuzzy model * Padres elevation fuzzy model * each of the four Padres 2080 climate fuzzy models

3. Set a habitat suitability threshold of .5 and create habitat suitability DataArrays from each climate model DataArray.

4. Calculate the suitable percentage of each suitability DataArray.


</span>

<span style='color:purple'>

1. Create function to use trapezoidal membership function on each DataArray:

</span>

In [3]:
def create_trapmf(da,
                  tol_min,
                  opt_min,
                  opt_max,
                  tol_max
                  ):
    """
    Use the sci-kit fuzzy trapezoidal membership function generator
    to put the values of a DataArray into a trapezoidal membership function.

    Parameters
    ----------
    da : DataArray
        The DataArray a trapezoidal membership function is being created for
    tol_min : int or float
        the minimum value of the tolerance range of the variable
    opt_min : int or float
        the minimum value of the optimum range of the variable
    opt_max : int or float
        the maximum value of the optimum range of the variable
    tol_max : int or float
        the maximum value of the tolerange range of the variable

    Returns
    -------
    fuzz_da : DataArray
        A DataArray with the shape and coordinates of the
        `da` but values created by the sci-kit fuzzy
        trapezoidal membership function generator
    """
    # save the shape of the DataArray
    da_shape = (da
                # .values leaves just an array, removes the xarray wrappers
                .values
                .shape)

    # copy DataArray to keep all coordinates, etc
    fuzz_da = da.copy()

    # create membership function
    fuzz_da.values = (
        # reshape 1D array created below back to da_shape
        np.reshape(
            fuzz.trapmf(
                # .flatten lays 2D array into 1D array
                da.values.flatten(),
                # variable tolerance and optimum values
                [tol_min, opt_min, opt_max, tol_max]),
                da_shape
                )
        )
    
    # name the fuzz_da based on the name of the DataArray parameter
    fuzz_da.name = f'{da.name} Fuzzy'
    return fuzz_da

In [4]:
# empty list for fuzzy DataArrays
fuzz_das_list = []

# for each DataArray in the das_list,
for i in tqdm(reproj_das_list):
    # if it is a soil DataArray,
    if 'Soil' in i.name:
        # use the create_trapmf fxn w/ pH optimum/tolerance values
        soil_fuzz_da = create_trapmf(i, 5.0, 6.0, 7.0, 7.5)
        fuzz_das_list.append(soil_fuzz_da)
    # if it is an elevation DataArray,
    if 'Elevation' in i.name:
        # use the create_trapmf fxn w/ elev optimum/tolerance values
        elev_fuzz_da = create_trapmf(i, 0, 100, 2500, 3000)
        fuzz_das_list.append(elev_fuzz_da)
    # if it is a precipitation DataArray,
    if 'Precipitation' in i.name:
        # use the create_trapmf fxn w/ precip optimum/tolerance values
        pr_fuzz_da = create_trapmf(i, 508, 1270, 1800, 5080)
        fuzz_das_list.append(pr_fuzz_da)

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
# check fuzz_das_list
# commented out to save memory
#fuzz_das_list

<span style='color:purple'>

2. Raster multiplication:

</span>

In [6]:
# multiply Siskiyou Soil pH Fuzzy,
# Siskiyou Elevation (m) Fuzzy, and
# Siskiyou Ave Annual Precip 2036-2065 climate models
s_2050_can = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[4]
s_2050_miroc = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[5]
s_2050_mri = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[6]
s_2050_gfdl = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[7]

In [7]:
# multiply Siskiyou Soil pH Fuzzy,
# Siskiyou Elevation (m) Fuzzy, and
# Siskiyou Ave Annual Precip 2066-2095 climate models
s_2080_can = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[8]
s_2080_miroc = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[9]
s_2080_mri = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[10]
s_2080_gfdl = fuzz_das_list[0] * fuzz_das_list[1] * fuzz_das_list[11]

In [8]:
# multiply Padres Soil pH Fuzzy,
# Padres Elevation (m) Fuzzy, and
# Padres Ave Annual Precip 2036-2065 climate models
p_2050_can = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[12]
p_2050_miroc = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[13]
p_2050_mri = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[14]
p_2050_gfdl = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[15]

In [9]:
# multiply Padres Soil pH Fuzzy,
# Padres Elevation (m) Fuzzy, and
# Padres Ave Annual Precip 2066-2095 climate models
p_2080_can = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[16]
p_2080_miroc = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[17]
p_2080_mri = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[18]
p_2080_gfdl = fuzz_das_list[2] * fuzz_das_list[3] * fuzz_das_list[19]

In [10]:
# add names to mutliplied rasters
s_2050_can.name = 's_2050_can'
s_2050_miroc.name = 's_2050_miroc'
s_2050_mri.name = 's_2050_mri'
s_2050_gfdl.name = 's_2050_gfdl'
s_2080_can.name = 's_2080_can'
s_2080_miroc.name = 's_2080_miroc'
s_2080_mri.name = 's_2080_mri'
s_2080_gfdl.name = 's_2080_gfdl'
p_2050_can.name = 'p_2050_can'
p_2050_miroc.name = 'p_2050_miroc'
p_2050_mri.name = 'p_2050_mri'
p_2050_gfdl.name = 'p_2050_gfdl'
p_2080_can.name = 'p_2080_can'
p_2080_miroc.name = 'p_2080_miroc'
p_2080_mri.name = 'p_2080_mri'
p_2080_gfdl.name = 'p_2080_gfdl'

<span style='color:purple'>

3. Set a habitat suitability threshold of .5 and create habitat suitability DataArrays from each climate model DataArray:

</span>

In [11]:
# list of multiplied rasters
raster_products = [
    s_2050_can,
    s_2050_miroc,
    s_2050_mri,
    s_2050_gfdl,
    s_2080_can,
    s_2080_miroc,
    s_2080_mri,
    s_2080_gfdl,
    p_2050_can,
    p_2050_miroc,
    p_2050_mri,
    p_2050_gfdl,
    p_2080_can,
    p_2080_miroc,
    p_2080_mri,
    p_2080_gfdl
]

In [12]:
# empty list for the suitability DataArrays
suit_das = []

# for each raster product,
for raster in tqdm(raster_products):
    # find the raster suitability 
    raster_suit = raster > .5
    # add a name to the raster_suit DataArray
    raster_suit.name = f'{raster.name}'
    # add the raster_suit DataArray to the suit_das list
    suit_das.append(raster_suit)

# check list
suit_das

  0%|          | 0/16 [00:00<?, ?it/s]

[<xarray.DataArray 's_2050_can' (y: 3622, x: 3989)> Size: 14MB
 array([[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]])
 Coordinates:
     band         int64 8B 1
     spatial_ref  int64 8B 0
   * x            (x) float64 32kB -124.4 -124.4 -124.4 ... -123.3 -123.3 -123.3
   * y            (y) float64 29kB 42.89 42.89 42.89 42.89 ... 41.88 41.88 41.88
     crs          int64 8B 0,
 <xarray.DataArray 's_2050_miroc' (y: 3622, x: 3989)> Size: 14MB
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [Fa

<span style='color:purple'>

4. Calculate the suitable percentage of each suitability DataArray.

I used chatgpt to write the code below.

I will calculate the percentage of 1s (pixels that are suitable) of each suitability DataArray.

</span>

In [13]:
# empty list for suitable percents
suitable_percents = []

# for each suitability DataArray,
for suit_da in tqdm(suit_das):
    # calculate the percent of the DataArray that is a 1
    percent_ones = (suit_da == 1).sum() / suit_da.size * 100
    # name the percent_ones DataArray
    percent_ones.name = f'{suit_da.name} percent suitable'
    # add the percent_ones DataArray to the suitable_percents list
    suitable_percents.append(percent_ones)

# check list
suitable_percents

  0%|          | 0/16 [00:00<?, ?it/s]

[<xarray.DataArray 's_2050_can percent suitable' ()> Size: 8B
 array(42.5948415)
 Coordinates:
     band         int64 8B 1
     spatial_ref  int64 8B 0
     crs          int64 8B 0,
 <xarray.DataArray 's_2050_miroc percent suitable' ()> Size: 8B
 array(44.82801891)
 Coordinates:
     band         int64 8B 1
     spatial_ref  int64 8B 0
     crs          int64 8B 0,
 <xarray.DataArray 's_2050_mri percent suitable' ()> Size: 8B
 array(41.91787631)
 Coordinates:
     band         int64 8B 1
     spatial_ref  int64 8B 0
     crs          int64 8B 0,
 <xarray.DataArray 's_2050_gfdl percent suitable' ()> Size: 8B
 array(42.69776812)
 Coordinates:
     band         int64 8B 1
     spatial_ref  int64 8B 0
     crs          int64 8B 0,
 <xarray.DataArray 's_2080_can percent suitable' ()> Size: 8B
 array(42.72003393)
 Coordinates:
     band         int64 8B 1
     spatial_ref  int64 8B 0
     crs          int64 8B 0,
 <xarray.DataArray 's_2080_miroc percent suitable' ()> Size: 8B
 array(44.9415

<span style='color:purple'>

Compare suitability percentages for each climate model for each national forest for each time period.

</span>

In [14]:
# Siskiyou CanESM2
print(f'Siskiyou 2050 CanESM2 suitability percentage is {suitable_percents[0].values}%')
print(f'Siskiyou 2080 CanESM2 suitability percentage is {suitable_percents[4].values}%')

# Siskiyou MIROC-ESM-CHEM
print(f'Siskiyou 2050 MIROC-ESM-CHEM suitability percentage is {suitable_percents[1].values}%')
print(f'Siskiyou 2080 MIROC-ESM-CHEM suitability percentage is {suitable_percents[5].values}%')

# Siskiyou MRI-CGCM3
print(f'Siskiyou 2050 MRI-CGCM3 suitability percentage is {suitable_percents[2].values}%')
print(f'Siskiyou 2080 MRI-CGCM3 suitability percentage is {suitable_percents[6].values}%')

# Siskiyou GFDL-ESM2M
print(f'Siskiyou 2050 GFDL-ESM2M suitability percentage is {suitable_percents[3].values}%')
print(f'Siskiyou 2080 GFDL-ESM2M suitability percentage is {suitable_percents[7].values}%')

Siskiyou 2050 CanESM2 suitability percentage is 42.59484150159488%
Siskiyou 2080 CanESM2 suitability percentage is 42.72003393096892%
Siskiyou 2050 MIROC-ESM-CHEM suitability percentage is 44.82801890732369%
Siskiyou 2080 MIROC-ESM-CHEM suitability percentage is 44.94156279298718%
Siskiyou 2050 MRI-CGCM3 suitability percentage is 41.91787631336811%
Siskiyou 2080 MRI-CGCM3 suitability percentage is 42.023059271638644%
Siskiyou 2050 GFDL-ESM2M suitability percentage is 42.69776811687691%
Siskiyou 2080 GFDL-ESM2M suitability percentage is 43.72059746301224%


In [15]:
# Los Padres CanESM2
print(f'Los Padres 2050 CanESM2 suitability percentage is {suitable_percents[8].values}%')
print(f'Los Padres 2080 CanESM2 suitability percentage is {suitable_percents[12].values}%')

# Los Padres MIROC-ESM-CHEM
print(f'Los Padres 2050 MIROC-ESM-CHEM suitability percentage is {suitable_percents[9].values}%')
print(f'Los Padres 2080 MIROC-ESM-CHEM suitability percentage is {suitable_percents[13].values}%')

# Los Padres MRI-CGCM3
print(f'Los Padres 2050 MRI-CGCM3 suitability percentage is {suitable_percents[10].values}%')
print(f'Los Padres 2080 MRI-CGCM3 suitability percentage is {suitable_percents[14].values}%')

# Los Padres GFDL-ESM2M
print(f'Los Padres 2050 GFDL-ESM2M suitability percentage is {suitable_percents[11].values}%')
print(f'Los Padres 2080 GFDL-ESM2M suitability percentage is {suitable_percents[15].values}%')

Los Padres 2050 CanESM2 suitability percentage is 6.686083674205136%
Los Padres 2080 CanESM2 suitability percentage is 8.823332847642842%
Los Padres 2050 MIROC-ESM-CHEM suitability percentage is 1.334569129354383%
Los Padres 2080 MIROC-ESM-CHEM suitability percentage is 1.1187083337775623%
Los Padres 2050 MRI-CGCM3 suitability percentage is 5.599121680078437%
Los Padres 2080 MRI-CGCM3 suitability percentage is 4.721274368928128%
Los Padres 2050 GFDL-ESM2M suitability percentage is 1.8210518298882872%
Los Padres 2080 GFDL-ESM2M suitability percentage is 1.8874270625162635%


In [15]:
%store suit_das

Stored 'suit_das' (list)


In [16]:
%store suitable_percents

Stored 'suitable_percents' (list)
